# Debugging in DSC2

## Intermediate scripts
Under the hood, DSC2 generates R, Python or Shell scripts based on computational routines provided in `exec` entries and executes these scripts to complete the computational tasks. There are often roughly two types of errors that may occur:

* There is an error from user provided computational routine
* There is an inconsistency between DSC script interface and user provided computational routine.

In case of errors DSC will fail. However the automatically generated script will be saved to disk and DSC2 will point you to the script in question so that you can debug interactively. Here we look at an example from [a benchmark on application of Bayesian Linear Model in genomic prediction](https://github.com/stephenslab/dsc2-blm).

In [1]:
! dsc -x blm.dsc -j8

INFO: DSC script exported to blm.html
INFO: Constructing DSC from blm.dsc ...
INFO: Building execution graph ...
DSC:  90%|█████████ | 9/10 [00:00<00:00, 10.97it/s]
ERROR: Failed to execute workflow DSC
[score_1 ['dsc_blm/datamaker.R_1_gemma_bs] RuntimeError:
	Failed to process statement sos_run('core_score_1:1', outp...ignature)\n: Failed to execute workflow core_score_1
[core_score_1_1 (score.R)] RuntimeError:
	Failed to execute script (ret=1).
Please use command
	Rscript \
	  --default-packages=datasets,methods,utils,stats,grDevices,graphics \
	  /home/gaow/GIT/lab-dsc/dsc2-blm/.sos/core_score_1_1_6.R
under "/home/gaow/GIT/lab-dsc/dsc2-blm" to test it.
[core_score_1] RuntimeError:
	1 failed step: core_score_1_1 (score.R)
[DSC] RuntimeError:
	1 failed step: score_1


The error message is a bit verbose but the part most relevant to debugging are hightlighted in green color on your terminal. Towards the end of the error message you should see that the problematic script is saved to `.sos/core_score_1_1_6.R`. You can also see that a file called `dsc_blm.transcript.html` is generated. 

## Debug with the problematic script
As prompted in the error message, let's try to execute `.sos/core_score_1_1_6.R`:

In [2]:
## r script UUID: dcfc55ef9ae7c9eab6397eefe425afc0
## BEGIN code by DSC2
DSC_LIBPATH <- NULL
source("/home/gaow/GIT/lab-dsc/dsc2-blm/.sos/.dsc/utils.R")
DSC_714A6C66AA <- list()
input.files <- c('dsc_blm/datamaker.R_7.rds', 'dsc_blm/datamaker.R_7_gemma_bslmm_1.rds')
for (i in 1:length(input.files)) DSC_714A6C66AA <- DSC_LMERGE(DSC_714A6C66AA, readRDS(input.files[i]))
meta <- DSC_714A6C66AA$meta
prediction <- DSC_714A6C66AA$prediction
## END code by DSC2
score = function(meta, prediction){
  true.value = meta$true.value
  mse = mean((true.value-prediction)^2)
  rmse = sqrt(mean((true.value-prediction)^2))
  pcor = cor(prediction, true.value)
  slope = lm(true.value~prediction)$coef[2]
  return(list(mse=mse, rmse=rmse, pcor=pcor, slope=slope))
}
output = score(meta, prediction)
## BEGIN code by DSC2
saveRDS(list(mse=output$mse, pcor=output$pcor, slope=output$slope), 'dsc_blm/datamaker.R_7_gemma_bslmm_1_score.R_1.rds')
## END code by DSC2

Warning message in true.value - prediction:
“longer object length is not a multiple of shorter object length”Warning message in true.value - prediction:
“longer object length is not a multiple of shorter object length”

ERROR: Error in cor(prediction, true.value): incompatible dimensions


You see that the script was propagated by DSC2 with variable assignment and input / output specifications. Also this script failed because of `incompatible dimensions` as indidcated by R. So what's going on? The function call takes two variables, `meta` and `prediction`. Let's examine what these variables look like:

In [3]:
source("/home/gaow/GIT/lab-dsc/dsc2-blm/.sos/.dsc/utils.R")
DSC_714A6C66AA <- list()
input.files <- c('dsc_blm/datamaker.R_7.rds', 'dsc_blm/datamaker.R_7_gemma_bslmm_1.rds')
for (i in 1:length(input.files)) DSC_714A6C66AA <- DSC_LMERGE(DSC_714A6C66AA, readRDS(input.files[i]))
meta <- DSC_714A6C66AA$meta
prediction <- DSC_714A6C66AA$prediction
print(meta)
print(paste("length meta: ", length(meta)))
print(prediction)
print(paste("length prediction: ", length(prediction)))


$true.value
       3881        3889       39359       41471       41484       41686 
 0.99831761  2.33609688  1.49504952  0.74828619  0.28221675  0.40967975 
      41800       41804       42274       62633       64869       78690 
 0.68604098 -0.45643508  0.44940658  0.38118361  0.52587135 -0.72295826 
      82710       83123       85599       85676       85711       85741 
-0.05990794  1.29932283  0.52996782  1.25576829  0.51239290  0.62052186 
      85786       96619      107059      133128      143058      143808 
 1.44805003  0.70355076  1.25938426  0.32484286  0.63805606  0.03753599 
     143989      217562      217777      218794      220384      292703 
 1.07711159 -0.48228439  0.04524029  0.15653729 -0.76760412  0.64203852 
     295146      295191      320849      321614      321822      321833 
 1.00396960  0.10798228 -0.68283238 -0.15248985  0.86163551  1.02641469 
     321840      323172      342294      343215      353782      359354 
 1.09444220  1.15157292  0.68958366  0.

It looks like `meta$true.value` has a lot more elements than `prediction`. Therefore the statement `mse = mean((true.value-prediction)^2)` must fail. We know from the [DSC script](https://github.com/stephenslab/dsc2-blm/blob/master/blm.dsc) that since we have set `test_size = 100` we do expect both `prediction` and `true.value` to be of length 100. So there must be something wrong with `prediction`, that somehow it gets truncated. This variable comes from `dsc_blm/datamaker.R_7_gemma_bslmm_1.rds` so there is nothing we can do in this script to fix the problem. 

However in practice, perhaps you run into error in your DSC benchmark that can obviously be identified and fixed in this script. You should fix it, test it out interactively, and if it works, you should apply your patch to the original scripts that DSC calls because fixing these propagated intermediate scripts is not enough. In this example the `score` function seems fine so there is nothing to do to the [`score.R` script](https://github.com/stephenslab/dsc2-blm/blob/master/score.R). 

## Debugging with the entire transcript profile
Now we want to find out which command has generated this set of problematic `prediction` data. That is, we want to find the command that generated `dsc_blm/datamaker.R_7_gemma_bslmm_1.rds` from which `prediction` is loaded. However the problematic script above did not tell us where this data comes from. This leads us to examining the entire transcript of the DSC benchmark, written in `dsc_blm.transcript.html`.

You can open up this file in your web browser, and *search* (`ctrl-F`) for the phrase `dsc_blm/datamaker.R_7_gemma_bslmm_1.rds`. This leads you to the following (this is a screenshot of a chunk of `dsc_blm.transcript.html`): 

![debug-1](../../img/debug-tips-1.png)

Unsurprisingly, running the code by itself produces no error message -- this is expected because otherwise DSC would have failed at this stage. Usually one should start interacting with this new piece of code, fixing it until the output looks good, and apply the patch to the original benchmark code, and finally run DSC again.

For this example, what exactly is the problem that led to truncated result? Well, the issue is not very apparent unless you are aware of the parallel proessing nature of DSC and familiar with the use of DSC `File()` operator. However this is a very common mistake one can make and this is why we want to show it here. Please take a look at line 7 of the R code:

```r
result <- 'gemma.out'
```

This is code automatically generated by DSC, based on your DSC script. In the DSC script the corresponding section is:

```
gemma_bslmm(bayeslasso):
  exec: gemma.bslmm.wrapper.R
  .alias: gemma_bslmm
  params:
    result: gemma.out
    .alias: args = List(w = nIter, s = burnin, result = result)

```

However it turns out here `gemma.out` is used as temporary file prefix for intermediate results from `gemma`, a command line program. It is directly called from shell, see Line 27, 37 and 42 of the R code. In practice what happens is all parallel DSC instances all creates / writes on the **same file** `gemma.out`, creating a I/O race. Therefore the file would have been corrupted, leading to truncated file.

To fix the problem one should use `File()` operator for such cases:

```
gemma_bslmm(bayeslasso):
  exec: gemma.bslmm.wrapper.R
  .alias: gemma_bslmm
  params:
    result: File()
    .alias: args = List(w = nIter, s = burnin, result = result)
```

This is the 2nd type of error listed in the beginning of this document, ie, an error in communication between DSC and your script. After fixing the error you can run DSC again:

In [4]:
! dsc -x blm.dsc -j8

INFO: DSC script exported to blm.html
INFO: Constructing DSC from blm.dsc ...
INFO: Building execution graph ...
Running core_gemma_bslmm_1 (gemma_bslmm) (00:02:00): 

DSC: 100%|██████████| 10/10 [02:19<00:00,  7.25s/it]
INFO: Building output database dsc_blm.rds ...
INFO: DSC complete!
INFO: Elapsed time 140.619 seconds.


You see that all `gemma_bslmm` related codes are re-executed and the DSC benchmark completed without an issue.

## Log files
The run-time information are kept in greater detail in `*.log` file. For the example above you should see `dsc_blm.log` generated for the DSC run. You can look into the log file for more run-time information to help performing diagnostics.